## Assignment-01 Solutions

下面是一些必要的函数

In [1]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])


def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])


def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])


def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest:
        return (seg_pat, saying), len(saying)

    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i

    return (seg_pat, saying), len(saying)


fail = [True, None]


def pat_match_with_seg(pattern, saying):
    if not pattern or not saying:
        return []

    pat = pattern[0]

    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail


def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}


def subsitite(rule, parsed_rules):
    if not rule:
        return []

    return [parsed_rules.get(rule[0], rule[0])] + subsitite(
        rule[1:], parsed_rules)

In [2]:
#我们给大家一些例子:

rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I am ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

### 问题1

编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [3]:
import random

In [4]:
def get_response(saying, response_rules):
    if not saying or not response_rules:
        return []
    for rule, responses in response_rules.items():
        pattern = pat_match_with_seg(rule.split(), saying.split())
        if pattern[-1] is None or len(pattern[-1][1]) == len(saying.split()):
            continue
        pattern = pat_to_dict(pattern)
        return ' '.join(subsitite(random.choice(responses).split(), pattern))

In [5]:
saying = "I am Andy"
get_response(saying, rules)

'Were you really Andy ?'

### 问题2

改写以上程序，将程序变成能够支持中文输入的模式。 提示: 你可以需用用到 jieba 分词

In [6]:
import jieba

pattern

In [7]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': [
        'what would it mean if you got ?y', 'Why do you want ?y',
        'Suppose you got ?y soon'
    ],
    '?*x if ?*y': [
        'Do you really think its likely that ?y', 'Do you wish that ?y',
        'What do you think about ?y', 'Really-- if ?y'
    ],
    '?*x no ?*y': [
        'why not?', 'You are being a negative',
        'Are you saying \'No\' just to be negative?'
    ],
    '?*x I am ?*y': [
        'Were you really', 'Perhaps I already knew you were ?y',
        'Why do you tell me you were ?y now?'
    ],
    '?*x I feel ?*y':
    ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': [
        '?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不',
        '我看你就像?y'
    ],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y': ['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y':
    ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y':
    ['我听到你这么说， 也很难过', '不应该让你这么难过的，你觉得你拥有什么，就会不难过?', '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗': ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y': ['为什么你觉得我是?y'],
    '?*x因为?*y': ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [8]:
alpha = ['?', '*', 'x', 'y', 'z']


def regular(rule_list):
    rule = []
    tmp = ''
    for item in rule_list:
        if item in alpha:
            tmp += item
        else:
            if len(tmp) > 0:
                rule.append(tmp)
                tmp = ''
            rule.append(item)
    if len(tmp) > 0:
        rule.append(tmp)
    return rule

In [9]:
def zh_get_response(saying, response_rules):
    """
    支持中文和英文
    """
    if not saying or not response_rules:
        return []
    seg = ' '
    for rule, responses in response_rules.items():
        if len(saying.split()[0]) == len(saying):
            # param saying is Chinese
            seg = ''
            if len(rule.split()[0]) != len(rule):
                continue  # if rule is English, continue
            pattern = pat_match_with_seg(
                regular(jieba.lcut(rule)), jieba.lcut(saying))
            if pattern[-1] is None or len(pattern[-1][1]) == len(
                    jieba.lcut(saying)):
                continue
            pattern = pat_to_dict(pattern)
            response = random.choice(responses)
            response = regular(jieba.lcut(response))
        else:
            seg = ' '
            # param saying is English
            if len(rule.split()[0]) == len(rule):
                continue  # if rule is Chinese, continue
            pattern = pat_match_with_seg(rule.split(), saying.split())
            if pattern[-1] is None or len(pattern[-1][1]) == len(
                    saying.split()):
                continue
            pattern = pat_to_dict(pattern)
            response = random.choice(responses).split()

        return seg.join(subsitite(response, pattern))

In [10]:
saying = '我想要度假'
zh_get_response(saying, rule_responses)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\唐观根\AppData\Local\Temp\jieba.cache
Loading model cost 0.867 seconds.
Prefix dict has been built succesfully.


'想问你，你觉得度假有什么意义呢?'

In [11]:
saying = "I am Andy"
zh_get_response(saying, rule_responses)

'Were you really'

### 问题3

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

答：可以设计不同的对话模式：闲聊、导购、或者前段时间很火的夸夸群对话模式

### 问题4

1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？ 
    
    答：这是一个典型的基于规则的程序，这种程序**不依赖标注数据、不需要训练、程序运行时间相对高效**。缺点是需要**耗费大量时间进行手工构建规则，这种程序通常是面向特定数据，因此缺乏学习能力**。
    
    
2. 什么是数据驱动？数据驱动在这个程序里如何体现？
    
    答：以数据为基础寻找数据规律，再通过程序解决实际问题。在这个程序里面，根据输入字符串在预定义模板中进行匹配，根据规则替换变量、再返回相应的回答就是数据驱动吧
    

3. 数据驱动与 AI 的关系是什么？ 

    答：深度学习是由数据驱动的，没有数据就没有深度学习，也就没有AI如今疯狂的发展速度。
